<a href="https://colab.research.google.com/github/LucianoBV/Procesamiento-del-habla/blob/main/Desaf%C3%ADo_Data_set_grupo_de_noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# busquemos el conjunto de datos de 20 grupos de noticias y comencemos a analizar los documentos.
newsgroups = fetch_20newsgroups(subset='all')
newsgroups.target_names[:5], len(newsgroups.data)

(['alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware'],
 18846)

In [ ]:
import random

# Seleccionamos 5 documentos al azar
random.seed(42)  # Fijamos la semilla para obtener resultados reproducibles
random_docs_indices = random.sample(range(len(newsgroups.data)), 5)
random_docs = [newsgroups.data[i] for i in random_docs_indices]


In [ ]:
# Vectorizamos el texto de todos los documentos
vectorizer = TfidfVectorizer(stop_words='english')
X_tfidf = vectorizer.fit_transform(newsgroups.data)

In [ ]:
# Calculamos la similitud de coseno entre los 5 documentos seleccionados y el resto
similarity_matrix = cosine_similarity(X_tfidf[random_docs_indices], X_tfidf)

In [ ]:
# Para cada documento, encontramos los 5 documentos más similares (excluyendo el propio documento)
most_similar_docs = [(-similarity_matrix[i, :]).argsort()[1:6] for i in range(5)]


In [ ]:
# Creamos un resumen con los documentos seleccionados y sus 5 más similares
similarity_summary = []
for i, doc_idx in enumerate(random_docs_indices):
    doc_info = {
        'Selected Document Index': doc_idx,
        'Selected Document Category': newsgroups.target_names[newsgroups.target[doc_idx]],
        'Most Similar Documents Indices': most_similar_docs[i],
        'Most Similar Documents Categories': [newsgroups.target_names[newsgroups.target[idx]] for idx in most_similar_docs[i]],
    }
    similarity_summary.append(doc_info)

In [ ]:
print(doc_info)

{'Selected Document Index': 7314, 'Selected Document Category': 'rec.motorcycles', 'Most Similar Documents Indices': array([ 1142, 13291, 13151,   785, 11568]), 'Most Similar Documents Categories': ['rec.motorcycles', 'misc.forsale', 'rec.motorcycles', 'rec.autos', 'talk.politics.guns']}


In [ ]:
# Vamos a entrenar los modelos Naïve Bayes (MultinomialNB y ComplementNB) y optimizar su rendimiento

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Separar en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, newsgroups.target, test_size=0.2, random_state=42)

# Entrenamos el modelo Multinomial Naive Bayes
multinomial_nb = MultinomialNB()
multinomial_nb.fit(X_train, y_train)
y_pred_mnb = multinomial_nb.predict(X_test)
f1_mnb = f1_score(y_test, y_pred_mnb, average='macro')

# Entrenamos el modelo Complement Naive Bayes
complement_nb = ComplementNB()
complement_nb.fit(X_train, y_train)
y_pred_cnb = complement_nb.predict(X_test)
f1_cnb = f1_score(y_test, y_pred_cnb, average='macro')
print(f1_mnb, f1_cnb)

0.8663399374130707 0.8964060472437889


In [ ]:
# Estudiemos la similaridad entre palabras tomando manualmente 5 palabras del vocabulario del dataset
# Primero obtendremos las palabras más frecuentes del vocabulario para seleccionar palabras relevantes

# Obtener el vocabulario y la frecuencia de las palabras
word_counts = np.asarray(X_tfidf.sum(axis=0)).flatten()
vocab = vectorizer.get_feature_names_out()

# Crear un dataframe con las palabras y sus frecuencias
word_freq_df = pd.DataFrame({'Word': vocab, 'Frequency': word_counts}).sort_values(by='Frequency', ascending=False)

# Seleccionaremos palabras comunes y relevantes manualmente para evitar términos poco interpretables
selected_words = ['computer', 'windows', 'game', 'science', 'car']

# Encontramos los índices de estas palabras en el vocabulario
selected_word_indices = [np.where(vocab == word)[0][0] for word in selected_words]

# Calculamos la similitud de coseno entre las palabras seleccionadas y el resto del vocabulario
word_similarity_matrix = cosine_similarity(X_tfidf.T[selected_word_indices], X_tfidf.T)

# Para cada palabra, encontramos las 5 más similares
most_similar_words = [(-word_similarity_matrix[i, :]).argsort()[1:6] for i in range(5)]

# Creamos un resumen con las palabras seleccionadas y sus 5 más similares
word_similarity_summary = []
for i, word in enumerate(selected_words):
    word_info = {
        'Selected Word': word,
        'Most Similar Words': [vocab[idx] for idx in most_similar_words[i]]
    }
    word_similarity_summary.append(word_info)


In [ ]:
# Mostrar el resumen de similaridad entre palabras
word_similarity_summary

[{'Selected Word': 'computer',
  'Most Similar Words': ['science',
   'lines',
   'subject',
   'organization',
   'edu']},
 {'Selected Word': 'windows',
  'Most Similar Words': ['dos', 'ms', 'os', 'file', 'run']},
 {'Selected Word': 'game',
  'Most Similar Words': ['games', 'espn', 'baseball', 'hockey', 'score']},
 {'Selected Word': 'science',
  'Most Similar Words': ['computer', 'uhunix', 'scientific', 'uhcc', 'cs']},
 {'Selected Word': 'car',
  'Most Similar Words': ['cars', 'mileage', 'dealer', 'saftey', 'engine']}]

Se seleccionaron 5 documentos al azar y se midió su similitud con el resto de los documentos.
Se entrenaron dos modelos de clasificación Naïve Bayes: MultinomialNB y ComplementNB. El modelo ComplementNB mostro un mejor resultado con un F1-score de 0.896, superando al MultinomialNB que obtuvo un F1-score de 0.866.
Al analizar la similitud entre 5 palabras seleccionadas computer, windows, game, science, y car, se encontró que las palabras más similares tienen una relación sclara. Por ejemplo, game está relacionado con términos como games, baseball, y hockey.